### <center>CRF<center>

"""
学习问题中，利用极大似然估计，估计P(Y|X)
预测问题中，利用给定的序列x，求出条件概率P(Y|X)最大的输出序列y
"""
"""在一个条件随机场里面计算标签序列的log-likelihood"""
##### crf_log_likelihood(inputs,tag_indices,sequence_lengths,transition_params=None)

- viterbi_decode 和 crf_decode 实现了相同功能，前者是numpy的实现，后者是 tensor 的实现，本文为了验证两者的解码结果是一致的。
https://blog.csdn.net/baobao3456810/article/details/83388516

In [4]:
from tensorflow.contrib.crf import crf_log_likelihood

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [33]:
TIME_STEPS = 15  # 20 # backpropagation through time 的time_steps
BATCH_SIZE = 1  # 50
INPUT_SIZE = 1  # x数据输入size
LR = 0.05  # learning rate
num_tags = 2

In [44]:
# 定义一个生成数据的 get_batch function:
def get_batch():
    xs = np.array([[2, 3, 4, 5, 5, 5, 1, 5, 3, 2, 5, 5, 5, 3, 5]])
    res = np.array([[0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1]])
    return [xs[:, :, np.newaxis], res]

xs,res = get_batch()
len(x)

15

In [35]:
# 定义 CRF 的主体结构
class CRF(object):
    def __init__(self, n_steps, input_size, num_tags, batch_size):
        self.n_steps = n_steps
        self.input_size = input_size
        self.num_tags = num_tags
        self.batch_size = batch_size
        self.xs = tf.placeholder(tf.float32, [None, self.n_steps, self.input_size], name='xs')
        self.ys = tf.placeholder(tf.int32, [self.batch_size, self.n_steps], name='ys')
        # 将输入 batch_size x seq_length x input_size   映射到 batch_size x seq_length x num_tags

        # Compute unary scores from a linear layer.
        weights = tf.get_variable("weights", [self.input_size, self.num_tags])
        matricized_x_t = tf.reshape(self.xs, [-1, self.input_size])
        matricized_unary_scores = tf.matmul(matricized_x_t, weights)
        self.unary_scores = tf.reshape(matricized_unary_scores, [self.batch_size, self.n_steps, self.num_tags])  # 势函数

        self.sequence_lengths = np.full(self.batch_size, self.n_steps, dtype=np.int32)

        log_likelihood, self.transition_params = crf_log_likelihood(self.unary_scores, self.ys, self.sequence_lengths)
        # add a training op to tune the parameters.
        self.cost = tf.reduce_mean(-log_likelihood) # 最小化对数似然函数
        self.train_op = tf.train.AdamOptimizer(LR).minimize(self.cost)
        
        # 直接使用crf_decode 在计算图中推理
        self.pred, viterbi_score = tf.contrib.crf.crf_decode(self.unary_scores, self.transition_params, self.sequence_lengths)
        

In [52]:
def get_feed_dict(model,xs,res):
    feed_dict = {
        model.xs: xs,
        model.ys: res,
    }
    # zip argument #3 must support iteration
#     seq_lengths = np.full(BATCH_SIZE, TIME_STEPS, dtype=np.int32)
    seq_lengths = [15]
    return feed_dict,seq_lengths

In [55]:
# train_for_viterbi():
tf.reset_default_graph()
# 搭建 CRF 模型
model = CRF(TIME_STEPS, INPUT_SIZE, num_tags, BATCH_SIZE)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# matplotlib可视化
plt.ion()  # 设置连续 plot
plt.show()
# 训练多次
for i in range(150):
    xs, res = get_batch()  # 提取 batch data
    # 初始化 data
    feed_dict,seq_lengths = get_feed_dict(model,xs,res)
    # 训练
#     _, cost = sess.run([model.train_op, model.cost],feed_dict=feed_dict)
    tf_unary_scores, tf_transition_params, _  = sess.run([model.unary_scores, model.transition_params, 
                                                                           model.train_op],
                                                                          feed_dict=feed_dict)
    # run-evluate 
    if i % 10 == 0:
        correct_labels = 0
        total_labels = 0
        for tf_unary_scores_, y_, sequence_length_ in zip(tf_unary_scores, res,
                                                          seq_lengths):
            # Remove padding from the scores and tag sequence.
            tf_unary_scores_ = tf_unary_scores_[:sequence_length_]
            y_ = y_[:sequence_length_]

            # Compute the highest scoring sequence.
            viterbi_sequence, _ = tf.contrib.crf.viterbi_decode(
              tf_unary_scores_, tf_transition_params)

            # Evaluate word-level accuracy.
            correct_labels += np.sum(np.equal(viterbi_sequence, y_))
            total_labels += sequence_length_
        accuracy = 100.0 * correct_labels / float(total_labels)
        print("Accuracy: %.2f%%" % accuracy)
    

Accuracy: 66.67%
Accuracy: 73.33%
Accuracy: 73.33%
Accuracy: 86.67%
Accuracy: 86.67%
Accuracy: 93.33%
Accuracy: 100.00%
Accuracy: 100.00%
Accuracy: 100.00%
Accuracy: 100.00%
Accuracy: 100.00%
Accuracy: 100.00%
Accuracy: 100.00%
Accuracy: 100.00%
Accuracy: 100.00%


In [23]:
tf.reset_default_graph()
# 搭建 CRF 模型
model = CRF(TIME_STEPS, INPUT_SIZE, num_tags, BATCH_SIZE)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# matplotlib可视化
plt.ion()  # 设置连续 plot
plt.show()
# 训练多次
for i in range(150):
    xs, res = get_batch()  # 提取 batch data
    # print(res.shape)
    # 初始化 data
    feed_dict = {
        model.xs: xs,
        model.ys: res,
    }
    # 训练
    _, cost, pred = sess.run(
        [model.train_op, model.cost, model.pred],
        feed_dict=feed_dict)
#     print(pred)
    
    # run-evluate
    
    

    # plotting

#     x = xs.reshape(-1, 1)
#     r = res.reshape(-1, 1)
#     p = pred.reshape(-1, 1)

#     x = range(len(x))
#     plt.clf()
#     plt.plot(x, r, 'r', x, p, 'b--')
#     plt.ylim((-1.2, 1.2))
#     plt.draw()
#     plt.pause(0.3)  # 每 0.3 s 刷新一次
    
#     # 打印 cost 结果
#     if i % 20 == 0:
#         print('cost: ', round(cost, 4))

31.874107
27.02692
22.591496
18.77221
15.783321
13.748077
12.615074
12.170368
12.134438
12.268688
12.421555
12.516518
12.523972
12.439838
12.273415
12.041
11.762436
11.459057
11.151756
10.859512
10.59766
10.37648
10.200424
10.068031
9.972931
9.905384
9.854378
9.809288
9.761318
9.7040825
9.633858
9.549393
9.4515
9.342476
9.225632
9.104652
8.983196
8.864365
8.750549
8.643158
8.542629
8.448528
8.359684
8.274567
8.1914215
8.108665
8.025017
7.939636
7.852192
7.762848
7.672127
7.5808105
7.4898224
7.4000473
7.312191
7.226757
7.1439705
7.063835
6.98608
6.9103317
6.8361435
6.7631187
6.6908417
6.619156
6.5479164
6.4772186
6.407154
6.3379936
6.2699165
6.2031403
6.137783
6.0738907
6.0114746
5.9504356
5.890625
5.8319283
5.7741814
5.7172966
5.661167
5.605789
5.551159
5.4972916
5.444229
5.3919907
5.340618
5.290104
5.2403946
5.191498
5.1433563
5.0959015
5.0491295
5.0029755
4.9574585
4.9125366
4.8682327
4.824539
4.7815056
4.73909
4.6973267
4.6561813
4.615631
4.5757217
4.536392
4.4976196
4.4593887
4.421